In [1]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import numpy as np

In [3]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

text = data.Field(tokenize = 'spacy', 
                  tokenizer_language = 'en_core_web_sm',
                  batch_first = True)
label = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(text, label)

train_data, val_data = train_data.split(random_state = random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 246k/84.1M [00:00<00:39, 2.14MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:00<00:00, 85.3MB/s]


In [4]:
# Building  vocab and loading  pretrained word embeddings.

Max_Vocab_Size = 25_000

text.build_vocab(train_data, 
                 max_size = Max_Vocab_Size, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

label.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.40MB/s]                          
 99%|█████████▉| 397815/400000 [00:14<00:00, 29171.84it/s]

In [5]:
Batch_Size = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size = Batch_Size, 
    device = device)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):

    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, 
                                      embedding_dim, 
                                      padding_idx = pad_idx)
        
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        embedded = self.embedding(text)        
        embedded = embedded.unsqueeze(1)
                
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
                    
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
                
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))
            
        return self.fc(cat)

In [7]:
#Currently the CNN model can only use 3 different sized filters, but we can actually improve the code of our model 
#to make it more generic and take any number of filters. We do this by placing all of our convolutional layers 
#in a nn.ModuleList, a function used to hold a list of PyTorch nn.Module

class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                        
        embedded = self.embedding(text)        
        embedded = embedded.unsqueeze(1)
                
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
                            
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
              
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [9]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = n_filters, 
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        embedded = self.embedding(text)
        embedded = embedded.permute(0, 2, 1)
                
        conved = [F.relu(conv(embedded)) for conv in self.convs]
                    
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
                    
        return self.fc(cat)

In [10]:
INPUT_dim = len(text.vocab)
EMBEDDING_dim = 100
N_FILTERS = 100
FILTER_sizes = [3,4,5]
OUTPUT_dim = 1
DROPOUT = 0.5
PAD_idx = text.vocab.stoi[text.pad_token]

model = CNN(INPUT_dim, EMBEDDING_dim, N_FILTERS, FILTER_sizes, OUTPUT_dim, DROPOUT, PAD_idx)

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,620,801 trainable parameters


In [14]:
pretrained_embeddings = text.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2359,  0.3021,  0.2607,  ..., -0.0165, -0.8968,  0.0380],
        [-0.5088,  0.2616, -0.4246,  ...,  0.4455,  1.1573,  0.1265],
        [ 0.6273,  0.4286,  0.6129,  ..., -0.3205,  0.0943, -0.3572]])

In [16]:
UNK_idx = text.vocab.stoi[text.unk_token]

model.embedding.weight.data[UNK_idx] = torch.zeros(EMBEDDING_dim)
model.embedding.weight.data[PAD_idx] = torch.zeros(EMBEDDING_dim)

In [17]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [18]:
def binary_accuracy(preds, y):
    
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [19]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [20]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [22]:
NUM_epochs = 5

best_val_loss = float('inf')

for epoch in range(NUM_epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')



Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 0.653 | Train Acc: 61.50%
	 Val. Loss: 0.500 |  Val. Acc: 77.80%
Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 0.419 | Train Acc: 80.94%
	 Val. Loss: 0.362 |  Val. Acc: 84.73%
Epoch: 03 | Epoch Time: 0m 19s
	Train Loss: 0.302 | Train Acc: 87.17%
	 Val. Loss: 0.323 |  Val. Acc: 86.21%
Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 0.217 | Train Acc: 91.51%
	 Val. Loss: 0.322 |  Val. Acc: 86.65%
Epoch: 05 | Epoch Time: 0m 20s
	Train Loss: 0.153 | Train Acc: 94.40%
	 Val. Loss: 0.330 |  Val. Acc: 86.54%


In [23]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.341 | Test Acc: 85.36%


In [27]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [text.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [28]:
predict_sentiment(model, "This film is not good")

0.6121520400047302

In [29]:
predict_sentiment(model, "This film is amazing")

0.9910925030708313